In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#https://github.com/kevinscaria/InstructABSA/

In [ ]:
!pip install transformers
!pip install keybert #keybert
!pip install datasets
!pip install evaluate
!pip install sentencepiece
!pip install -U accelerate

## Restart

In [ ]:
!git clone https://github.com/kevinscaria/InstructABSA.git

In [ ]:
#import pandas as pd
#df_train_i = pd.read_json('/kaggle/input/annotationspyabsa/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result_English_Train.json',orient='records')
#df_train_i.head()

In [ ]:
#df_train_i.shape

In [ ]:
#df_valid_i = pd.read_json('/kaggle/input/annotationspyabsa/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result_English_Valid.json',orient='records')
#df_valid_i.head()

In [ ]:
#df_valid_i.shape

## KeyBert + Sentiment

In [ ]:
'''
from keybert import KeyBERT
kw_model = KeyBERT()
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
'''

In [ ]:
'''
final_train_data = []
for row in df_train_i.itertuples():
    polarity = []
    combine = []
    try:
        keywords = kw_model.extract_keywords(row.sentence)
        combine +=[i[0] for i in keywords]
        for aspect in [i[0] for i in keywords]:
            polarity += [classifier(row.sentence,  text_pair=aspect)[0]['label']]
        final_train_data.append([row.sentence,combine,polarity])
    except:
        print(row.sentence)
'''

In [ ]:
'''
final_valid_data = []
for row in df_valid_i.itertuples():
    polarity = []
    combine = []
    try:
        keywords = kw_model.extract_keywords(row.sentence)
        combine +=[i[0] for i in keywords]
        for aspect in [i[0] for i in keywords]:
            polarity += [classifier(row.sentence,  text_pair=aspect)[0]['label']]
        final_valid_data.append([row.sentence,combine,polarity])
    except:
        print(row.sentence)
'''

In [ ]:
#key_train = pd.DataFrame(final_train_data, columns =['sentence', 'aspect','polarity'])
#key_valid = pd.DataFrame(final_valid_data, columns =['sentence', 'aspect','polarity'])

In [ ]:
#key_train.head(1)

In [ ]:
#key_valid.head(1)

In [ ]:
#key_train.to_csv('key_train.csv',index=False)
#key_valid.to_csv('key_valid.csv',index=False)

In [5]:
import pandas as pd
df_train_i = pd.read_csv('/kaggle/input/kerybertaspect/key_train.csv')
df_valid_i = pd.read_csv('/kaggle/input/kerybertaspect/key_valid.csv')

## create a model with generate aspect with sentiment using same model

In [6]:
import os
import torch
use_mps = True if torch.has_mps else False
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from InstructABSA.InstructABSA.data_prep import DatasetLoader

In [7]:
class InstructionsHandler:
    def __init__(self):
        self.joint = {}
    def load_instruction_set(self, ):
        self.joint['bos_instruct'] = """Definition: The output will be the aspects (both implicit and explicit) and the aspects sentiment polarity. In cases where there are no aspects the output should be noaspectterm:none.
        Positive example 1-
        input: this hospital has a good team of doctors who will take care of all your needs brilliantly.
        output: doctors:positive
        Positive example 2- 
        input: Arthur as Irv at ham hospital ran an Nagar , Madurai has a doctor who engages you in a conversation and tries to take your mind off the pain and he has trained the staff to do so as well.
        output: doctor:positive, staff:positive
        Now complete the following example-
        input: """
        self.joint['delim_instruct'] = ''
        self.joint['eos_instruct'] = ' \noutput:'

In [3]:
import os
task_name = 'joint_task'
experiment_name = 'custom_absa'
model_checkpoint = 'allenai/tk-instruct-base-def-pos' #base model
#model_checkpoint = 'kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined' #pre-trained model
print('Experiment Name: ', experiment_name)
model_out_path = './Models'
model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}")
print('Model output path: ', model_out_path)

Experiment Name:  custom_absa
Model output path:  ./Models/joint_task/allenaitk-instruct-base-def-pos-custom_absa


In [9]:
df_train_i = df_train_i.rename(columns={'polarity': 'sentiment'})
df_valid_i = df_valid_i.rename(columns={'polarity': 'sentiment'})

In [10]:
import ast
df_train = df_train_i[['sentence','aspect','sentiment']]
df_train.rename(columns={'sentence': 'raw_text'}, inplace=True)
df_train['aspectTerms'] = ''
for i in df_train.itertuples():
    try:
        if len(i.aspect) > 0:
            k = [{'term': j[0],'polarity':j[1]} for j in list(zip(ast.literal_eval(i.aspect),ast.literal_eval(i.sentiment)))]
        else:
            k = [{'term':'noaspectterm', 'polarity':'none'}]
        df_train.at[i.Index, 'aspectTerms'] = k
    except Exception as e:
        print(e)
df_train = df_train[['raw_text','aspectTerms']]
df_train.head(1)

,raw_text,aspectTerms
0,everyone in this hospital from the staff to th...,"[{'term': 'patients', 'polarity': 'Positive'},..."


In [11]:
df_valid = df_valid_i[['sentence','aspect','sentiment']]
df_valid.rename(columns={'sentence': 'raw_text'}, inplace=True)
df_valid['aspectTerms'] = ''
for i in df_valid.itertuples():
    try:
        if len(i.aspect) > 0:
            k = [{'term': j[0],'polarity':j[1]} for j in list(zip(ast.literal_eval(i.aspect),ast.literal_eval(i.sentiment)))]
        else:
            k = [{'term':'noaspectterm', 'polarity':'none'}]
        df_valid.at[i.Index, 'aspectTerms'] = k
    except Exception as e:
        print(e)
df_valid = df_valid[['raw_text','aspectTerms']]
df_valid.head(1)

,raw_text,aspectTerms
0,the staff the nurses and the doctors are highl...,"[{'term': 'gora', 'polarity': 'Positive'}, {'t..."


In [12]:
df_valid.shape

(4239, 2)

In [13]:
df_valid.tail(2)

,raw_text,aspectTerms
4237,the doctors at this hospital have at least yea...,"[{'term': 'surgery', 'polarity': 'Neutral'}, {..."
4238,I underwent a very complicated surgery at ran ...,"[{'term': 'surgery', 'polarity': 'Negative'}, ..."


In [14]:
#testing dataset
df_test = df_valid[0:1000]

In [15]:
df_test.shape

(1000, 2)

In [16]:
df_valid = df_valid[1000:]

In [17]:
df_valid.shape

(3239, 2)

In [18]:
# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(df_train, df_test, df_valid)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_aspe_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct'], instruct_handler.joint['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_aspe_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct'], instruct_handler.joint['eos_instruct'])
if loader.val_df_id is not None:
    loader.val_df_id = loader.create_data_in_aspe_format(loader.val_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct'], instruct_handler.joint['eos_instruct'])

In [32]:
import numpy as np
import sys
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from transformers import (
    DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments, Trainer, Seq2SeqTrainer
)


class T5Generator:
    def __init__(self, model_checkpoint):
        self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
        self.data_collator = DataCollatorForSeq2Seq(self.tokenizer)
        self.device = 'cuda' if torch.has_cuda else ('mps' if torch.has_mps else 'cpu')

    def tokenize_function_inputs(self, sample):
        """
        Udf to tokenize the input dataset.
        """
        model_inputs = self.tokenizer(sample['text'], max_length=512, truncation=True)
        labels = self.tokenizer(sample["labels"], max_length=64, truncation=True)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
        
    def train(self, tokenized_datasets, **kwargs):
        """
        Train the generative model.
        """
        #Set training arguments
        args = Seq2SeqTrainingArguments(
            **kwargs
        )

        # Define trainer object
        trainer = Seq2SeqTrainer(
            self.model,
            args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["validation"] if tokenized_datasets.get("validation") is not None else None,
            tokenizer=self.tokenizer,
            data_collator=self.data_collator,
        )
        print("Trainer device:", trainer.args.device)

        # Finetune the model
        torch.cuda.empty_cache()
        print('\nModel training started ....')
        trainer.train()

        # Save best model
        trainer.save_model()
        return trainer

    def get_labels(self, tokenized_dataset, batch_size = 4, max_length = 512, sample_set = 'train'):
        """
        Get the predictions from the trained model.
        """
        def collate_fn(batch):
            input_ids = [torch.tensor(example['input_ids']) for example in batch]
            input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
            return input_ids
        dataloader = DataLoader(tokenized_dataset[sample_set], batch_size=batch_size, collate_fn=collate_fn)
        predicted_output = []
        predict_probability = []
        self.model.to(self.device)
        print('Model loaded to: ', self.device)

        for batch in tqdm(dataloader):
            batch = batch.to(self.device)
            output_ids = self.model.generate(batch, max_length = max_length)
            output_ids_prob = self.model.generate(batch,return_dict_in_generate=True, output_scores=True,top_p=.9, max_length=max_length)
            transition_scores = self.model.compute_transition_scores(output_ids_prob.sequences, output_ids_prob.scores, normalize_logits=True)
            predict_probability += [np.exp(transition_scores.cpu().numpy())[i].sum(axis=0)/len(transition_scores[i]) for i in range(len(transition_scores))]
            output_texts = self.tokenizer.batch_decode(output_ids, skip_special_tokens=True)
            for output_text in output_texts:
                predicted_output.append(output_text)
        return predicted_output, predict_probability
    
    def get_metrics(self, y_true, y_pred, is_triplet_extraction=False):

        total_pred = 0
        total_gt = 0
        tp = 0

        if not is_triplet_extraction:
            for gt, pred in zip(y_true, y_pred):
                gt_list = gt.split(', ')
                pred_list = pred.split(', ')
                total_pred+=len(pred_list)
                total_gt+=len(gt_list)
                for gt_val in gt_list:
                    for pred_val in pred_list:
                        if pred_val in gt_val or gt_val in pred_val:
                            tp+=1
                            break

        else:
            for gt, pred in zip(y_true, y_pred):
                gt_list = gt.split(', ')
                pred_list = pred.split(', ')
                total_pred+=len(pred_list)
                total_gt+=len(gt_list)
                for gt_val in gt_list:
                    gt_asp = gt_val.split(':')[0]

                    try:
                        gt_op = gt_val.split(':')[1]
                    except:
                        continue

                    try:
                        gt_sent = gt_val.split(':')[2]
                    except:
                        continue

                    for pred_val in pred_list:
                        pr_asp = pred_val.split(':')[0]

                        try:
                            pr_op = pred_val.split(':')[1]
                        except:
                            continue

                        try:
                            pr_sent = gt_val.split(':')[2]
                        except:
                            continue

                        if pr_asp in gt_asp and pr_op in gt_op and gt_sent == pr_sent:
                            # print(gt_asp, pr_asp, ' --> ', gt_op, pr_op, ' --> ', gt_sent, pr_sent)
                            tp+=1

        p = tp/total_pred
        r = tp/total_gt
        return p, r, 2*p*r/(p+r), None

In [20]:
# Create T5 utils object
t5_exp = T5Generator(model_checkpoint)
# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [21]:
id_ds

DatasetDict({
    train: Dataset({
        features: ['raw_text', 'aspectTerms', 'labels', 'text', '__index_level_0__'],
        num_rows: 16956
    })
    test: Dataset({
        features: ['raw_text', 'aspectTerms', 'labels', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['raw_text', 'aspectTerms', 'labels', 'text'],
        num_rows: 3239
    })
})

In [22]:
id_tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['raw_text', 'aspectTerms', 'labels', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 16956
    })
    test: Dataset({
        features: ['raw_text', 'aspectTerms', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['raw_text', 'aspectTerms', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 3239
    })
})

In [23]:
# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'lr_scheduler_type':'cosine',
    'per_device_train_batch_size':4,
    'per_device_eval_batch_size':4,
    'num_train_epochs':8,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps,
    'fp16_full_eval':True
}

In [24]:
# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

Trainer device: cuda:0

Model training started ....


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.150800,0.112104
2,0.103700,0.074400
3,0.074500,0.065034
4,0.065900,0.062676
5,0.059700,0.058017
6,0.050700,0.056635
7,0.046500,0.056589
8,0.046300,0.056924


## Evaluation

In [39]:
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - Training set   
id_tr_pred_labels, id_tr_pred_prob = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - Validation set
id_te_pred_labels, id_te_pred_prob = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'validation', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['validation']['labels']]

# Get prediction labels - UnseenTest set
id_test_pred_labels, id_test_pred_prob = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_test_labels = [i.strip() for i in id_ds['test']['labels']]

  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

Model loaded to:  cuda


100%|██████████| 1060/1060 [30:50<00:00,  1.75s/it]


Model loaded to:  cuda


100%|██████████| 203/203 [05:53<00:00,  1.74s/it]


Model loaded to:  cuda


100%|██████████| 63/63 [01:50<00:00,  1.75s/it]


In [ ]:
len(id_tr_pred_prob)

In [ ]:
id_tr_pred_labels

In [ ]:
id_tr_labels

In [ ]:
with open('actual_train_labels.txt','w',encoding='utf-8') as f:
    f.write(str(id_tr_labels))

In [ ]:
id_tr_pred_labels

In [ ]:
id_te_labels

In [ ]:
with open('actual_test_labels.txt','w',encoding='utf-8') as f:
    f.write(str(id_te_labels))

In [ ]:
id_te_pred_labels

In [ ]:
with open('predicted_test_labels.txt','w',encoding='utf-8') as f:
    f.write(str(id_te_pred_labels))

In [40]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)
p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
p, r, f1, _ = t5_exp.get_metrics(id_test_labels, id_test_pred_labels)
print('UnseenTest Precision: ', p)
print('UnseenTest Recall: ', r)
print('UnseenTest F1: ', f1)

Train Precision:  0.9552227554840818
Train Recall:  0.9552107544443746
Train F1:  0.9552167549265339
Test Precision:  0.9300919842312746
Test Recall:  0.9301530981010578
Test F1:  0.9301225401622918
UnseenTest Precision:  0.9329632792485055
UnseenTest Recall:  0.9329632792485055
UnseenTest F1:  0.9329632792485054


In [ ]:
id_tr_pred_labels[0]

In [ ]:
id_tr_labels[0].split(',')

In [41]:
check = pd.DataFrame(id_ds['train']['raw_text'], columns=['clean_text'])

In [42]:
check['actual_label'] = id_tr_labels
check['actual_aspects'] = [str([i.split(':')[0] for i in id_tr_labels[j].split(',')]) for j in range(len(id_tr_labels))]
check['actual_sentiments'] = [str([i.split(':')[1] for i in id_tr_labels[j].split(',')]) for j in range(len(id_tr_labels))]
check['model_text_generated_label'] = id_tr_pred_labels
check['predicted_aspects'] = [str([i.split(':')[0] for i in id_tr_pred_labels[j].split(',')]) for j in range(len(id_tr_pred_labels))]
check['predicted_sentiments'] = [str([i.split(':')[1] for i in id_tr_pred_labels[j].split(',')]) for j in range(len(id_tr_pred_labels))]
check['model_text_generate_probability']=id_tr_pred_prob

In [43]:
check['dataset_type'] = ['train']*check.shape[0]

In [ ]:
check.shape

In [ ]:
check.head()

In [57]:
valid_polarity = []
for j in range(len(id_te_labels)):
    try:
        valid_polarity+=[str([i.split(':')[1] for i in id_te_labels[j].split(',')])]
    except:
        valid_polarity+=[""]

In [58]:
len(valid_polarity)

3239

In [51]:
[str([i.split(':')[1] if for i in id_te_labels[j].split(',')]) for j in range(len(id_te_labels))]

IndexError: list index out of range

In [59]:
check_valid = pd.DataFrame(id_ds['validation']['raw_text'], columns=['clean_text'])
check_valid['actual_label'] = id_te_labels
check_valid['actual_aspects'] = [str([i.split(':')[0] for i in id_te_labels[j].split(',')]) for j in range(len(id_te_labels))]
check_valid['actual_sentiments'] = valid_polarity
check_valid['model_text_generated_label'] = id_te_pred_labels
check_valid['predicted_aspects'] = [str([i.split(':')[0] for i in id_te_pred_labels[j].split(',')]) for j in range(len(id_te_pred_labels))]
check_valid['predicted_sentiments'] = [str([i.split(':')[1] for i in id_te_pred_labels[j].split(',')]) for j in range(len(id_te_pred_labels))]
check_valid['model_text_generate_probability']=id_te_pred_prob
check_valid['dataset_type'] = ['valid']*check_valid.shape[0]

In [45]:
check_test = pd.DataFrame(id_ds['test']['raw_text'], columns=['clean_text'])
check_test['actual_label'] = id_test_labels
check_test['actual_aspects'] = [str([i.split(':')[0] for i in id_test_labels[j].split(',')]) for j in range(len(id_test_labels))]
check_test['actual_sentiments'] = [str([i.split(':')[1] for i in id_test_labels[j].split(',')]) for j in range(len(id_test_labels))]
check_test['model_text_generated_label'] = id_test_pred_labels
check_test['predicted_aspects'] = [str([i.split(':')[0] for i in id_test_pred_labels[j].split(',')]) for j in range(len(id_test_pred_labels))]
check_test['predicted_sentiments'] = [str([i.split(':')[1] for i in id_test_pred_labels[j].split(',')]) for j in range(len(id_test_pred_labels))]
check_test['model_text_generate_probability']=id_test_pred_prob
check_test['dataset_type'] = ['test']*check_test.shape[0]

In [60]:
final = pd.concat([check.reset_index(drop=True),check_valid.reset_index(drop=True),check_test.reset_index(drop=True)])

In [61]:
final.reset_index(drop=True, inplace=True)
final.tail()

,clean_text,actual_label,actual_aspects,actual_sentiments,model_text_generated_label,predicted_aspects,predicted_sentiments,model_text_generate_probability,dataset_type
21190,Jaya Laxmi heart care seem okay l Madurai has ...,"madurai:Positive, jaya:Neutral, doctors:Positi...","['madurai', ' jaya', ' doctors', ' heart', ' l...","['Positive', 'Neutral', 'Positive', 'Neutral',...","madurai:Positive, jaya:Neutral, laxmi:Neutral,...","['madurai', ' jaya', ' laxmi', ' heart', ' doc...","['Positive', 'Neutral', 'Neutral', 'Neutral', ...",0.898708,test
21191,Medi spring hospital and research center Rusta...,"pharmacy:Positive, medicines:Neutral, gora:Neu...","['pharmacy', ' medicines', ' gora', ' hospital...","['Positive', 'Neutral', 'Neutral', 'Neutral', ...","pharmacy:Neutral, medicines:Neutral, gora:Neut...","['pharmacy', ' medicines', ' gora', ' hospital...","['Neutral', 'Neutral', 'Neutral', 'Neutral', '...",0.665091,test
21192,"Arthur as Irv at ham hospital ran an Nagar , M...","madurai:Positive, nagar:Neutral, hospital:Neut...","['madurai', ' nagar', ' hospital', ' arthur', ...","['Positive', 'Neutral', 'Neutral', 'Positive',...","madurai:Positive, nagar:Neutral, hospital:Neut...","['madurai', ' nagar', ' hospital', ' doctors',...","['Positive', 'Neutral', 'Neutral', 'Positive',...",0.950015,test
21193,the doctors at muck he ej a clinic full char G...,"gora:Positive, surgery:Positive, doctors:Posit...","['gora', ' surgery', ' doctors', ' clinic', ' ...","['Positive', 'Positive', 'Positive', 'Neutral'...","gora:Positive, surgery:Positive, clinic:Neutra...","['gora', ' surgery', ' clinic', ' doctors', ' ...","['Positive', 'Positive', 'Neutral', 'Positive'...",0.933928,test
21194,this hospital has cutting edge equipment and a...,"hospital:Positive, equipment:Positive, staff:P...","['hospital', ' equipment', ' staff', ' trained...","['Positive', 'Positive', 'Positive', 'Positive...","hospital:Positive, equipment:Positive, staff:P...","['hospital', ' equipment', ' staff', ' trained...","['Positive', 'Positive', 'Positive', 'Positive...",0.975610,test


In [62]:
final.to_excel('key_result.xlsx',index=False)

In [63]:
final.to_csv('key_result.csv',index=False)

## Testing

In [27]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import warnings
warnings.filterwarnings('ignore')

In [64]:
tokenizer = AutoTokenizer.from_pretrained(model_out_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_out_path)

bos_instruction = """Definition: The output will be the aspects (both implicit and explicit) and the aspects sentiment polarity. In cases where there are no aspects the output should be noaspectterm:none.
    Positive example 1-
    input: this hospital has a good team of doctors who will take care of all your needs brilliantly.
    output: doctors:positive
    Positive example 2- 
    input: Arthur as Irv at ham hospital ran an Nagar , Madurai has a doctor who engages you in a conversation and tries to take your mind off the pain and he has trained the staff to do so as well.
    output: doctor:positive, staff:positive
    Now complete the following example-
    input: """
delim_instruct = ''
eos_instruct = ' \noutput:'
text = 'This hospital has doctors who gave me the best consultation even though it was not what other doctors would say. I was completely fine now because i followed thier advice.'
tokenized_text = tokenizer(bos_instruction + text + delim_instruct + eos_instruct, return_tensors="pt")
output = model.generate(tokenized_text.input_ids,max_length=512)
print('Model output: ', tokenizer.decode(output[0], skip_special_tokens=True))

Model output:  hospital:Positive, consultation:Positive, doctors:Positive, best:Positive, probably:Positive


In [66]:
text = 'Nobody caring here, i never suggest this hospital.'
tokenized_text = tokenizer(bos_instruction + text + delim_instruct + eos_instruct, return_tensors="pt")
output = model.generate(tokenized_text.input_ids,max_length=512)
print('Model output: ', tokenizer.decode(output[0], skip_special_tokens=True))

Model output:  hospital:Negative, suggest:Negative, care:Negative, hurt:Negative
